In [1]:
import pandas as pd

pdf = pd.read_csv(f"labelled_newscatcher_dataset.csv", sep=";")
pdf["id"] = pdf.index

In [3]:
from sentence_transformers import InputExample

pdf_subset = pdf.head(1000)

def example_create_fn(doc1: pd.Series) -> InputExample:
    """
    Helper function that outputs a sentence_transformer guid, label, and text
    """
    return InputExample(texts=[doc1])

faiss_train_examples = pdf_subset.apply(
    lambda x: example_create_fn(x["title"]), axis=1
).tolist()

In [4]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "all-MiniLM-L6-v2"
)
faiss_title_embedding = model.encode(pdf_subset.title.values.tolist())
len(faiss_title_embedding), len(faiss_title_embedding[0])

(1000, 384)

In [6]:
import numpy as np
import faiss

pdf_to_index = pdf_subset.set_index(["id"], drop=False)
id_index = np.array(pdf_to_index.id.values).flatten().astype("int")

content_encoded_normalized = faiss_title_embedding.copy()
faiss.normalize_L2(content_encoded_normalized) # we do this step in order to make the dot product equal to cosine similarity

# Index1DMap translates search results to IDs: https://faiss.ai/cpp_api/file/IndexIDMap_8h.html#_CPPv4I0EN5faiss18IndexIDMapTemplateE
# The IndexFlatIP below builds index
index_content = faiss.IndexIDMap(faiss.IndexFlatIP(len(faiss_title_embedding[0]))) #IndexFlatIP means no vector compression is involved
#IndexIDMap is merely a function to maintain mapping between your IDs and embeddings
index_content.add_with_ids(content_encoded_normalized, id_index)

#### We define a search function below to first vectorize our query text, and then search for the vectors with the closest distance.



In [7]:
def search_content(query, pdf_to_index, k=3):
    query_vector = model.encode([query])
    faiss.normalize_L2(query_vector)

    # We set k to limit the number of vectors we want to return
    top_k = index_content.search(query_vector, k)
    ids = top_k[1][0].tolist()
    similarities = top_k[0][0].tolist()
    results = pdf_to_index.loc[ids]
    results["similarities"] = similarities
    return results

In [9]:
print(search_content("meteor", pdf_to_index))

       topic                                               link  \
id                                                                
294  SCIENCE  https://www.haveeru.com.mv/perseid-meteor-show...   
113  SCIENCE  https://www.goshennews.com/news/rachel-shenk-m...   
316  SCIENCE  https://www.firstpost.com/tech/science/perseid...   

             domain       published_date  \
id                                         
294  haveeru.com.mv  2020-08-10 21:13:55   
113  goshennews.com  2020-08-17 02:15:00   
316   firstpost.com  2020-08-12 03:17:12   

                                                 title lang   id  similarities  
id                                                                              
294  Perseid meteor shower 2020: How and when to lo...   en  294      0.590013  
113  RACHEL SHENK: Meteor shower is reminder of nat...   en  113      0.573253  
316  Perseid meteor shower peaks tonight: Interesti...   en  316      0.561288  
